In [1]:
import pandas as pd

data = pd.read_csv('spam.csv')
data.head(5)

,Category,Message
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."


In [2]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [3]:
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [0]:
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords


def rm_stopword(m):
  stop_words = set(stopwords.words('english'))
  words = word_tokenize(m)
  return [word for word in words if word not in stop_words]

data.Message = data.Message.apply(rm_stopword)

In [5]:
data.head(5)

,Category,Message
0,ham,"[Go, jurong, point, ,, crazy.., Available, bug..."
1,ham,"[Ok, lar, ..., Joking, wif, u, oni, ...]"
2,spam,"[Free, entry, 2, wkly, comp, win, FA, Cup, fin..."
3,ham,"[U, dun, say, early, hor, ..., U, c, already, ..."
4,ham,"[Nah, I, n't, think, goes, usf, ,, lives, arou..."


In [0]:
from nltk.stem import SnowballStemmer

def stemming(m):
  stemmer = SnowballStemmer('english')
  return [stemmer.stem(word) for word in m]

data['stemming'] = data.Message.apply(stemming)

In [7]:
data.head(5)

,Category,Message,stemming
0,ham,"[Go, jurong, point, ,, crazy.., Available, bug...","[go, jurong, point, ,, crazy.., avail, bugi, n..."
1,ham,"[Ok, lar, ..., Joking, wif, u, oni, ...]","[ok, lar, ..., joke, wif, u, oni, ...]"
2,spam,"[Free, entry, 2, wkly, comp, win, FA, Cup, fin...","[free, entri, 2, wkli, comp, win, fa, cup, fin..."
3,ham,"[U, dun, say, early, hor, ..., U, c, already, ...","[u, dun, say, earli, hor, ..., u, c, alreadi, ..."
4,ham,"[Nah, I, n't, think, goes, usf, ,, lives, arou...","[nah, i, n't, think, goe, usf, ,, live, around..."


In [8]:
from nltk.stem import WordNetLemmatizer 
nltk.download('wordnet')
def lemmatisation(m):
  lemmatizer = WordNetLemmatizer() 
  return [lemmatizer.lemmatize(word) for word in m]

data['Lemmatisation'] = data.Message.apply(lemmatisation)

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


In [0]:
data.head(5)

,Category,Message,stemming,Lemmatisation
0,ham,"[Go, jurong, point, ,, crazy.., Available, bug...","[go, jurong, point, ,, crazy.., avail, bugi, n...","[Go, jurong, point, ,, crazy.., Available, bug..."
1,ham,"[Ok, lar, ..., Joking, wif, u, oni, ...]","[ok, lar, ..., joke, wif, u, oni, ...]","[Ok, lar, ..., Joking, wif, u, oni, ...]"
2,spam,"[Free, entry, 2, wkly, comp, win, FA, Cup, fin...","[free, entri, 2, wkli, comp, win, fa, cup, fin...","[Free, entry, 2, wkly, comp, win, FA, Cup, fin..."
3,ham,"[U, dun, say, early, hor, ..., U, c, already, ...","[u, dun, say, earli, hor, ..., u, c, alreadi, ...","[U, dun, say, early, hor, ..., U, c, already, ..."
4,ham,"[Nah, I, n't, think, goes, usf, ,, lives, arou...","[nah, i, n't, think, goe, usf, ,, live, around...","[Nah, I, n't, think, go, usf, ,, life, around,..."


Le stemming supprime les prefixe et le suffixe du mot. Il permet de reduire un mot a sa racine.
Le lemmatisation consiste a reduire un mot a sa forme canonique. L'idee est de garder le sens des mots

In [0]:
#Transformer la variable stemming en chaine de caractere
data.stemming = data.stemming.apply(lambda x: " ".join(word for word in x) )

In [0]:
#Vu que le calcul de tfidf s'applique sur des chaines de caractere, On peut l'appliquer sur la variable stemming
from sklearn.feature_extraction.text import TfidfVectorizer
vectorizer = TfidfVectorizer()
v = vectorizer.fit_transform(data.stemming)


In [0]:
#Il ne reste plus qu'a creer une dataFrame avec les resultats precedents
df = pd.DataFrame(data=v.todense())

In [49]:
#On teste si ça s'est correctement deroulè
df.head(5)

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,...,7477,7478,7479,7480,7481,7482,7483,7484,7485,7486,7487,7488,7489,7490,7491,7492,7493,7494,7495,7496,7497,7498,7499,7500,7501,7502,7503,7504,7505,7506,7507,7508,7509,7510,7511,7512,7513,7514,7515,7516
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [0]:
#Je concatene notre nouvelle dataFrame avec la premiere
data = pd.concat([data, df], axis=1)

In [61]:
data.shape

(5572, 7521)

Je vais  encoder les **spam** et les **ham**

In [62]:
data.Category = data.Category.replace({'ham': 1, 'spam': 0})

TypeError: ignored

In [63]:
data.head(5)

,Category,Message,stemming,Lemmatisation,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,...,7477,7478,7479,7480,7481,7482,7483,7484,7485,7486,7487,7488,7489,7490,7491,7492,7493,7494,7495,7496,7497,7498,7499,7500,7501,7502,7503,7504,7505,7506,7507,7508,7509,7510,7511,7512,7513,7514,7515,7516
0,1,"[Go, jurong, point, ,, crazy.., Available, bug...","go jurong point , crazy.. avail bugi n great w...","[Go, jurong, point, ,, crazy.., Available, bug...",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,1,"[Ok, lar, ..., Joking, wif, u, oni, ...]",ok lar ... joke wif u oni ...,"[Ok, lar, ..., Joking, wif, u, oni, ...]",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0,"[Free, entry, 2, wkly, comp, win, FA, Cup, fin...",free entri 2 wkli comp win fa cup final tkts 2...,"[Free, entry, 2, wkly, comp, win, FA, Cup, fin...",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,1,"[U, dun, say, early, hor, ..., U, c, already, ...",u dun say earli hor ... u c alreadi say ...,"[U, dun, say, early, hor, ..., U, c, already, ...",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,1,"[Nah, I, n't, think, goes, usf, ,, lives, arou...","nah i n't think goe usf , live around though","[Nah, I, n't, think, go, usf, ,, life, around,...",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


Nous allons commencer l'apprentissage

In [66]:
#On splite d'abord les donnees en donnees d'apprentissage et donnees test
from sklearn.model_selection import train_test_split
#En prenant le temps de definir la variables cible et les variables d'apprentissage
target = 'Category'
features = [x for x in data.columns if x not in ['Category','Message','stemming','Lemmatisation']]
len(features)

7517

In [0]:
#Nous allons ensuite faire une partition des donnees: 70% d'apprentissage et 30% de test
X_train, X_test, y_train, y_test = train_test_split(
    data[features],
    data[target],
    test_size=0.3,
    random_state=42
)

Nous commencons par les algorithmes simples 
Si jamais les resultats ne sont pas satisfaisants on fait appel aux algorithmes gros calibres
Cette strategie nous permettra de pas depenser des ressources inutilement

In [69]:
#Pour cela on commence la regression logistique
from sklearn.linear_model import LogisticRegression
lr = LogisticRegression()
lr.fit(X_train, y_train)

/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='warn', n_jobs=None, penalty='l2',
                   random_state=None, solver='warn', tol=0.0001, verbose=0,
                   warm_start=False)

In [71]:
#On mesure les erreurs commises lors de l'apprentissage
from sklearn.metrics import f1_score, confusion_matrix
f1_score(lr.predict(X_train), y_train)

0.9822364589400115

Nous avons reussi a predire 98.22 % des resultats. On peut dire que c'est satisfaisant. Ainsi inutile de d'utiliser d'auutres algortihmes

In [73]:
#On verfier les resultats avec notre matrice de confusion
confusion_matrix(lr.predict(X_train), y_train)

array([[ 405,    4],
       [ 118, 3373]])